In [1]:
# Importing libraries
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import joblib
import pickle
df=pd.read_csv('imdb_sup.csv')

In [2]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [3]:
df = df.sample(frac = 1)
df.reset_index(inplace=True,drop=True)

In [4]:
df

,Review,Rating,Sentiment
0,Although its plot is taken from the history of...,4,0
1,This movie is based on a Stephen King novel in...,3,0
2,I usually read reviews before I watch a movie....,4,0
3,Luckily for Bill Murray this is such a light-w...,8,1
4,"MARY, MARY, BLOODY MARY is an OK time killer. ...",4,0
...,...,...,...
49995,I've seen this movie a few times and with each...,10,1
49996,"Considering it's basically low-budget cast, th...",7,1
49997,"This movie is a nice, cute family oriented fil...",10,1
49998,Mario Van Peebles pops up for less than a five...,1,0


In [5]:
df.drop(['Rating'],axis=1)

,Review,Sentiment
0,Although its plot is taken from the history of...,0
1,This movie is based on a Stephen King novel in...,0
2,I usually read reviews before I watch a movie....,0
3,Luckily for Bill Murray this is such a light-w...,1
4,"MARY, MARY, BLOODY MARY is an OK time killer. ...",0
...,...,...
49995,I've seen this movie a few times and with each...,1
49996,"Considering it's basically low-budget cast, th...",1
49997,"This movie is a nice, cute family oriented fil...",1
49998,Mario Van Peebles pops up for less than a five...,0


In [6]:
df=df.iloc[:1000]

In [7]:
df

,Review,Rating,Sentiment
0,Although its plot is taken from the history of...,4,0
1,This movie is based on a Stephen King novel in...,3,0
2,I usually read reviews before I watch a movie....,4,0
3,Luckily for Bill Murray this is such a light-w...,8,1
4,"MARY, MARY, BLOODY MARY is an OK time killer. ...",4,0
...,...,...,...
995,Emily Watson's Natalia is absolutely the most ...,10,1
996,This is an awesome action film with great one ...,9,1
997,it's movies like these that make you wish that...,10,1
998,This movie is spoofed in an episode of Mystery...,2,0


In [22]:
for i in df['Review'].iloc[:1]:
    print(i)

Although its plot is taken from the history of ancient Rome, 'Caligula' is not made in the style of sword-and-sandal epics like 'Spartacus' or 'Gladiator'. (That style of movie-making was out of favour in the late seventies). Rather, it more closely resembles a cross between a soft-core porn movie and a video nasty. At least, the shortened 150 minute version does. I have never seen the full 210 minute version, but to judge from the descriptions of it on this page, it would seem closer to a cross between a hard-core porn movie and a video nasty. <br /><br />As one would expect from a film produced by the publisher of 'Penthouse', there is much naked flesh to be seen, and the film lovingly catalogues Caligula's sexual perversions, including his incestuous affair with his sister Drusilla. What is perhaps unexpected from the publisher of a magazine so closely associated with the heterosexual male lifestyle is that there is as much male flesh as female on display and distinctly homoerotic o

In [37]:
df=df.drop(['Rating'],axis=1)

In [38]:
df

,Review,Sentiment
0,Although its plot is taken from the history of...,0
1,This movie is based on a Stephen King novel in...,0
2,I usually read reviews before I watch a movie....,0
3,Luckily for Bill Murray this is such a light-w...,1
4,"MARY, MARY, BLOODY MARY is an OK time killer. ...",0
...,...,...
995,Emily Watson's Natalia is absolutely the most ...,1
996,This is an awesome action film with great one ...,1
997,it's movies like these that make you wish that...,1
998,This movie is spoofed in an episode of Mystery...,0


In [39]:


corpus = []
count=0

# Looping till 1000 because the number of rows are 1000
for i in range(0, 1000):
    # Removing the special character from the reviews and replacing it with space character
    review = re.sub(pattern='[^a-zA-Z]', repl=' ', string=df['Review'][i])

    # Converting the review into lower case character
    review = review.lower()
    count+=1
    print(count)

    # Tokenizing the review by words
    review_words = review.split()

    # Removing the stop words using nltk stopwords
    review_words = [word for word in review_words if not word in set(
        stopwords.words('english'))]

    # Stemming the words
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review_words]

    # Joining the stemmed words
    review = ' '.join(review)

    # Creating a corpus
    corpus.append(review)






1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [18]:
print(type(corpus))
print(corpus[0])
print(corpus[1])

<class 'list'>
although plot taken histori ancient rome caligula made style sword sandal epic like spartacu gladiat style movi make favour late seventi rather close resembl cross soft core porn movi video nasti least shorten minut version never seen full minut version judg descript page would seem closer cross hard core porn movi video nasti br br one would expect film produc publish penthous much nake flesh seen film lovingli catalogu caligula sexual pervers includ incestu affair sister drusilla perhap unexpect publish magazin close associ heterosexu male lifestyl much male flesh femal display distinctli homoerot overton mani scene obsess sex balanc equal obsess violenc though bob guccion gore vidal tinto brass tri kill two taboo one stone remark coincid director brazen film call brass scriptwrit gori one call gore charact put death mutil varieti sadist way charm scene homosexu rape br br emperor caligula predecessor tiberiu certainli known debauch lifestyl film concentr sex violenc n

In [40]:
# Creating Bag of Words model
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:, 1].values

# Creating a pickle file for the CountVectorizer model
print(X[0])

[0 0 0 ... 0 0 0]


In [41]:
print(X.shape)
print(y.shape)
print(X[0])

(1000, 1500)
(1000,)
[0 0 0 ... 0 0 0]


In [42]:
print(y)

[0 0 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 1 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 0 0 1 1 1 0 1 0 0 1 1 1 1
 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0
 1 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0
 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1 1 1 0 1 0 0 0
 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 0
 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 0 0 1 0 1 1 0 1 1 0 1 0 1 1 1
 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 0
 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0
 0 1 0 1 1 1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 0 0 1 1 0 1 1 0 0 1 0
 0 1 0 1 1 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1
 1 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 0 1 0 1
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 1 0 1
 1 1 1 0 1 1 1 1 0 1 0 1 

In [51]:

# Model Building
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=0)

# Fitting Naive Bayes to the Training set
classifier = MultinomialNB(alpha=1)
classifier.fit(X_train, y_train)

# Creating a pickle file for the Multinomial Naive Bayes model


MultinomialNB(alpha=1)

In [28]:
print(X_test)

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [34]:
print(y_train)

[10  7  4  3  9 10  1  1 10 10  9  4  8  9 10  8  7  8  1  8  9  8  1  1
 10  1  9  4  7  3  1  1  2  2  4 10  3  9  3  9  8  1  2  9  8  4  4  4
 10  3  1  8 10  4 10  9  4  9  1  1 10  7  2  4  7  9  1  2  3  4  9  1
  3  9  7  2  3  8  7  7  3 10  8  9  8  1  1  3 10  1  9  9  1  2 10  1
  2  1  3  3  8  1  3  3  7  7  8  9  7 10  7  7 10  8  3  1  9  7 10 10
  7  7 10  1  9  8  7  3  8  7  4  4 10  7  4  1  9  9 10  1  8  4  1  4
  8 10  7 10  2  3  2 10 10  1  7  9 10  3  3  1  2 10  1  9  7 10  9  1
  4  1  3  9  7  2  1  8  1 10  1  1  2  1 10  7 10  9  8  9  2  3  2  3
  7 10  3  4  7  1  2  1  1  1  7  8  8  7 10  2  4  1 10  4  8  8  1  4
  3  4  8  9 10  7  3  4  1  7  8  9 10 10  2  1  1 10  3  1 10  7  1  7
  1  7  1  2  8  8 10  2 10 10  3  1 10  7  9  3  2  1 10 10  7  3  9  2
  3  7  8  4  3  3  8  2 10  7  9  4  3  1  1  2  4  7 10 10 10  7  1  4
  3  3  1  8  8  9  7  8  8  7 10 10  8 10  3 10 10  9  9  8  9 10  4 10
  2  7  8 10  2  9  4  1  2  2 10  7  4  1  8  1  1

In [52]:
yhat = classifier.predict(X_test)

In [33]:
print(yhat)

[ 1  4  9  8  9  9  7  8  9  1  1  1  1 10  4  3  4  1  8  8  1  7  8  2
  1  3  1 10  7  7  4  2  2  1  8  8  1  3  9  4  1  8  9  1 10 10  3  3
  7  7 10  8 10  8  1  9  7 10  7  7  1  7  2  3  2  1  1 10 10  1 10  1
  9  3  1  1  9  1  4  3 10  1  1  7  1  4 10  8 10  2  3  4  9  2  1  9
  1 10  1 10  4 10  8  1  1  1  4  3  1 10  1  1  4  1  2  3  1  4  9  1
  3  2  8  8  8  4  9  7  7 10  2 10  1 10  4  7  8 10 10  4  2 10 10  1
 10  9 10  1  9 10 10  1  3 10  7 10  4  1  8  7  7  7  4  8 10  1  7  4
  1  8  8 10  1  1 10  7  1  1  2  4 10  7  2  8  2  4  4  2 10 10  1  2
  8  2 10  8  1  8  1  7]


In [43]:
from sklearn.metrics import accuracy_score 

In [53]:
score = accuracy_score(y_test, yhat)  


In [54]:
print(score)

0.775
